In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm
from IPython.display import display, Markdown, Latex
import pickle

np.random.seed(100)

SMALL_SIZE = 12
MEDIUM_SIZE = 14
LARGE_SIZE = 16

plt.rcdefaults()
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams['font.size'] = MEDIUM_SIZE
plt.rcParams['axes.titlesize'] = LARGE_SIZE
plt.rcParams['axes.labelsize'] = MEDIUM_SIZE
plt.rcParams['xtick.labelsize'] = SMALL_SIZE
plt.rcParams['ytick.labelsize'] = SMALL_SIZE
plt.rcParams['legend.fontsize'] = MEDIUM_SIZE
plt.rcParams['figure.titlesize'] = LARGE_SIZE
plt.rcParams['figure.figsize'] = [7.2, 4.8]
plt.rcParams['figure.dpi'] = 60
plt.rcParams['figure.facecolor'] = (1.0, 1.0, 1.0, 0.0)

from utils import LREQ, SREQ

In [2]:
sreq_df = pd.read_csv("SREQ_examples.csv")
lreq_df = pd.read_csv("LREQ_examples.csv")

params_df = lreq_df.groupby(
    ['N','Y','alpha','beta','gamma','eta']
).agg(
    num_examples = ('delta','count')
).reset_index()

params_df = params_df.loc[
    params_df['num_examples']>1
].reset_index(drop=True)

In [3]:
examples_df = []

for idx, row in params_df.iterrows():
    params = dict(row).copy()
    N = params['N']
    Y = params['Y']
    alpha = params['alpha']
    beta = params['beta']
    gamma = params['gamma']
    eta = params['eta']
    
    lreq_sub_df = lreq_df.loc[
        (lreq_df['N']==N) &
        (lreq_df['Y']==Y) &
        (lreq_df['alpha']==alpha) & 
        (lreq_df['beta']==beta) &
        (lreq_df['gamma']==gamma) &
        (lreq_df['eta']==eta)
    ].sample(2).reset_index(drop=True)
    
    delta0 = lreq_sub_df.loc[0, 'delta']
    delta1 = lreq_sub_df.loc[1, 'delta']
    
    lreq0 = LREQ({'N':N,'Y':Y,'alpha':alpha,'beta':beta,'gamma':gamma,'delta':delta0,'eta':eta})
    lreq1 = LREQ({'N':N,'Y':Y,'alpha':alpha,'beta':beta,'gamma':gamma,'delta':delta1,'eta':eta})
    
    M0 = lreq0.sol['M']
    M1 = lreq1.sol['M']
    
    params_sr = {'N':N,'Y':Y,'M':M0,'alpha':alpha,'beta':beta,'gamma':gamma,'delta':delta1,'eta':eta}
    sreq = SREQ(params_sr)
    if sreq.check_solution():
        examples_df.append({
            'N':N,'Y':Y,'alpha':alpha,'beta':beta,'gamma':gamma,'delta0':delta0,'delta1':delta1,'eta':eta,'profit_sr':sreq.sol['total_profit']
        })

    params_sr = {'N':N,'Y':Y,'M':M1,'alpha':alpha,'beta':beta,'gamma':gamma,'delta':delta0,'eta':eta}
    sreq = SREQ(params_sr)
    if sreq.check_solution():
        examples_df.append({
            'N':N,'Y':Y,'alpha':alpha,'beta':beta,'gamma':gamma,'delta0':delta1,'delta1':delta0,'eta':eta,'profit_sr':sreq.sol['total_profit']
        })

examples_df = pd.DataFrame.from_dict(examples_df)
print(len(examples_df))

11153


In [4]:
examples_df.to_csv("CostShock_examples.csv", header=True, index=False)